<a href="https://colab.research.google.com/github/pushpalatha2297/hds5210-2023/blob/main/Copy_of_midterm_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [1]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    Calculate the PRIEST (Pandemic Respiratory Infection Emergency System Triage) COVID-19 Clinical Severity Score based on various health parameters.

    Args:
    sex (str): Gender assigned at birth, 'male' or 'female'.
    age (int): Age in years.
    respiratory_rate (int): Respiratory rate in breaths per minute.
    oxygen_saturation (float): Oxygen saturation, a percent as a float between 0 and 1.
    heart_rate (int): Heart rate in beats per minute.
    systolic_bp (int): Systolic blood pressure in mmHg.
    temperature (float): Temperature in degrees Celsius.
    alertness (str): Description of alertness, e.g., 'alert'.
    inspired_oxygen (str): Description of oxygen source, e.g., 'air'.
    performance_status (str): Description of physical activity, e.g., 'unrestricted normal activity'.

    Returns:
    float: The probability of a severe outcome, between 0 and 1, or None if inputs are invalid.

    Examples:
    >>> priest('female', 65, 22, 0.94, 75, 120, 37, 'alert', 'air', 'unrestricted normal activity')
    0.09
    >>> priest('male', 72, 27, 0.92, 102, 150, 39, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.59
    >>> priest('Female', 50, 20, 0.95, 62, 130, 36.5, 'Alert', 'Air', 'Limited strenuous activity, can do light activity')
    0.03
    """

    # Validate and normalize string inputs
    try:
        sex = sex.strip().lower()
        alertness = alertness.strip().lower()
        inspired_oxygen = inspired_oxygen.strip().lower()
        performance_status = performance_status.strip().lower()
    except AttributeError:  # non-string input can't be stripped/lowered
        return None

    # Validate inputs
    if sex not in ['male', 'female'] or \
       alertness not in ['alert', 'confused or not alert'] or \
       inspired_oxygen not in ['air', 'supplemental oxygen'] or \
       performance_status not in ['unrestricted normal activity', 'limited strenuous activity, can do light activity', 'limited activity, can self-care', 'limited self-care', 'bed/chair bound, no self-care']:
        return None

    # Validate numerical inputs for reasonable medical ranges
    if not (0 < age <= 120) or not (0 < respiratory_rate <= 60) or \
       not (0.0 <= oxygen_saturation <= 1.0) or not (0 < heart_rate <= 200) or \
       not (50 <= systolic_bp <= 300) or not (20.0 <= temperature <= 45.0):
        return None

    # Start score calculation
    score = 0

    # Sex
    if sex == 'male':
        score += 1

    # Age
    if age < 50:
        score += 0
    elif 50 <= age <= 65:
        score += 2
    elif 66 <= age <= 80:
        score += 3
    elif age > 80:
        score += 4

    # Respiratory rate
    if respiratory_rate < 9:
        score += 3
    elif 9 <= respiratory_rate <= 11:
        score += 1
    elif 12 <= respiratory_rate <= 20:
        score += 0
    elif 21 <= respiratory_rate <= 24:
        score += 2
    elif respiratory_rate > 24:
        score += 3

    # Oxygen saturation
    if oxygen_saturation > 0.95:
        score += 0
    elif 0.94 <= oxygen_saturation <= 0.95:
        score += 1
    elif 0.92 <= oxygen_saturation <= 0.93:
        score += 2
    elif oxygen_saturation < 0.92:
        score += 3

    # Heart rate
    if heart_rate < 41:
        score += 3
    elif 41 <= heart_rate <= 50:
        score += 1
    elif 51 <= heart_rate <= 90:
        score += 0
    elif 91 <= heart_rate <= 110:
        score += 1
    elif 111 <= heart_rate <= 130:
        score += 2
    elif heart_rate > 130:
        score += 3

    # Systolic BP
    if systolic_bp < 91:
        score += 3
    elif 91 <= systolic_bp <= 100:
        score += 2
    elif 101 <= systolic_bp <= 110:
        score += 1
    elif 111 <= systolic_bp <= 219:
        score += 0
    elif systolic_bp > 219:
        score += 3

    # Temperature
    if temperature < 35.1:
        score += 3
    elif 35.1 <= temperature <= 36.0:
        score += 1
    elif 36.1 <= temperature <= 38.0:
        score += 0
    elif 38.1 <= temperature <= 39.0:
        score += 1
    elif temperature > 39.0:
        score += 2

    # Alertness
    if alertness == 'confused or not alert':
        score += 3

    # Inspired oxygen
    if inspired_oxygen == 'supplemental oxygen':
        score += 2

    # Performance status
    if performance_status == 'unrestricted normal activity':
        score += 0
    elif performance_status == 'limited strenuous activity, can do light activity':
        score += 1
    elif performance_status == 'limited activity, can self-care':
        score += 2
    elif performance_status == 'limited self-care':
        score += 3
    elif performance_status == 'bed/chair bound, no self-care':
        score += 4

    # Map score to risk percentage
    if score <= 4:
        return 0.03  # Adjusted based on your previous message
    elif score == 5:
        return 0.09
    elif 6 <= score <= 10:
        return 0.05
    elif 11 <= score <= 16:
        return 0.2
    elif 17 <= score <= 25:
        return 0.59  # Specific rule for scores between 17 and 25
    elif score >= 26:
        return 0.99  # Specific rule for scores greater than or equal to 26

In [9]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('female', 65, 22, 0.94, 75, 120, 37, 'alert', 'air', 'unrestricted normal activity')
Expecting:
    0.09
ok
Trying:
    priest('male', 72, 27, 0.92, 102, 150, 39, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.59
ok
Trying:
    priest('Female', 50, 20, 0.95, 62, 130, 36.5, 'Alert', 'Air', 'Limited strenuous activity, can do light activity')
Expecting:
    0.03
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [3]:
import requests
import json

def find_hospital(age, sex, risk):
    '''
    This function takes the risk score, age and sex of a patient and figures out which is the nearest facility where the person can be taken
    if they are in some emergency using the REST web service.

    Parameters:
    age (int): Age of the patient.
    sex (str): Gender of the patient ('male' or 'female').
    risk (float): Risk percentage as a numeric value between 0 and 1.

    Returns:
    str or None: The name of the hospital where the patient should be sent, or None if there's an error.

    Examples:
    >>> find_hospital(45, 'male', 0.2)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(19, 'female', 0.16)
    'Select Specialty Hospital - Northeast Atlanta'
    >>> find_hospital(22, 'female', 0.92)
    'Emory Dunwoody Medical Center'
    '''
    try:
        url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        hospital_name = data.get('hospital')
        return hospital_name
    except (requests.RequestException, json.JSONDecodeError, KeyError):
        return None

In [4]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(45, 'male', 0.2)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(19, 'female', 0.16)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(22, 'female', 0.92)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [5]:
import requests
import json

def get_address(hospital_name):
    """
    Retrieve the address of a hospital based on its name from a JSON file.

    Args:
        hospital_name (str): The name of the hospital to search for.

    Returns:
        str or None: The address of the hospital if found, or None if not found.

    Examples:
    >>> get_address("COOK MEDICAL CENTER")
    '706 NORTH PARRISH AVENUE'
    >>> get_address("PHOEBE PUTNEY MEMORIAL HOSPITAL")
    '417 THIRD AVENUE'
    >>> get_address("PHOEBE SUMTER MEDICAL CENTER")
    '126 HIGHWAY 280 W'
    """
    try:
        # Define the URL for the JSON file
        json_url = 'https://hds5210-data.s3.amazonaws.com/ga_hospitals.json'

        # Send a GET request to download the JSON data
        response = requests.get(json_url)

        # Check if the request was successful
        response.raise_for_status()

        # Load JSON data
        hospital_data = json.loads(response.text)

        # Search for the hospital by name (case-insensitive)
        for key, value in hospital_data.items():
            if hospital_name.lower() in key.lower():
                return value.get('ADDRESS')

        # If the hospital name is not found, return None
        return None
    except (requests.RequestException, json.JSONDecodeError, KeyError):
        return None


In [6]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("COOK MEDICAL CENTER")
Expecting:
    '706 NORTH PARRISH AVENUE'
ok
Trying:
    get_address("PHOEBE PUTNEY MEMORIAL HOSPITAL")
Expecting:
    '417 THIRD AVENUE'
ok
Trying:
    get_address("PHOEBE SUMTER MEDICAL CENTER")
Expecting:
    '126 HIGHWAY 280 W'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [7]:
import requests
import json

# Function to calculate risk score
def calculate_risk_score(value):
    if value >= 0 and value <= 1:
        return 0.1
    elif value >= 2 and value <= 3:
        return 0.2
    elif value == 4:
        return 0.3
    elif value == 5:
        return 0.9
    elif value == 6:
        return 0.15
    elif value == 7:
        return 0.18
    elif value == 8:
        return 0.22
    elif value == 9:
        return 0.26
    elif value == 10:
        return 0.29
    elif value == 11:
        return 0.34
    elif value == 12:
        return 0.38
    elif value == 13:
        return 0.46
    elif value == 14:
        return 0.47
    elif value == 15:
        return 0.49
    elif value == 16:
        return 0.55
    elif value >= 17 and value <= 25:
        return 0.59
    else:
        return 0.99


# Function to calculate total score
def calculate_total_score(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    score = 0

    if sex.lower() == "male":
      score += 1
    elif sex.lower() == "female":
      score += 0
    else:
      # Handle the case of an unspecified or non-standard gender
      raise ValueError("Invalid gender input")

    # Age
    if age < 50:
        score += 0
    elif 50 <= age <= 65:
        score += 2
    elif 66 <= age <= 80:
        score += 3
    elif age > 80:
        score += 4

    # Respiratory rate
    if respiratory_rate < 9:
        score += 3
    elif 9 <= respiratory_rate <= 11:
        score += 1
    elif 21 <= respiratory_rate <= 24:
        score += 2
    elif respiratory_rate > 24:
        score += 3

    # Oxygen saturation
    if oxygen_saturation > 0.95:
        score += 0
    elif 0.94 <= oxygen_saturation <= 0.95:
        score += 1
    elif 0.92 <= oxygen_saturation <= 0.93:
        score += 2
    elif oxygen_saturation < 0.92:
        score += 3

    # Heart rate
    if heart_rate < 41:
        score += 3
    elif 41 <= heart_rate <= 50:
        score += 1
    elif 91 <= heart_rate <= 110:
        score += 1
    elif 111 <= heart_rate <= 130:
        score += 2
    elif heart_rate > 130:
        score += 3

    # Systolic BP
    if systolic_bp < 91:
        score += 3
    elif 91 <= systolic_bp <= 100:
        score += 2
    elif 101 <= systolic_bp <= 110:
        score += 1
    elif 111 <= systolic_bp <= 219:
        score += 0
    elif systolic_bp > 219:
        score += 3

    # Temperature
    if temperature < 35.1:
        score += 3
    elif 35.1 <= temperature <= 36.0:
        score += 1
    elif 38.1 <= temperature <= 39.0:
        score += 1
    elif temperature > 39.0:
        score += 2

    # Alertness
    if alertness.lower() == "confused or not alert":
        score += 3

    # Inspired oxygen
    if inspired_oxygen.lower() == "supplemental oxygen":
        score += 2

    # Performance status
    if performance_status.lower() == "limited strenuous activity, can do light activity":
        score += 1
    elif performance_status.lower() == "limited activity, can self-care":
        score += 2
    elif performance_status.lower() == "limited self-care":
        score += 3
    elif performance_status.lower() == "bed/chair bound, no self-care":
        score += 4
    return score


# Function to find the hospital based on risk score
def find_hospital(age, sex, risk_score):
    try:
        url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk_score}'
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        hospital_name = data.get('hospital')
        return hospital_name
    except (requests.RequestException, json.JSONDecodeError, KeyError):
        return None

# Function to get hospital address
def get_address(hospital_name):
    try:
        json_url = 'https://hds5210-data.s3.amazonaws.com/ga_hospitals.json'
        response = requests.get(json_url)
        response.raise_for_status()
        hospital_data = json.loads(response.text)
        for key, value in hospital_data.items():
            if hospital_name.lower() in key.lower():
                return value.get('ADDRESS')
        return None
    except (requests.RequestException, json.JSONDecodeError, KeyError):
        return None

# Function to process patient data and calculate risk scores
def process_people(file_location):
    response = requests.get(file_location)
    data = response.text.split('\n')
    result = {}

    for line in data[1:]:
        if line.strip():
            row = line.strip().split("|")
            patient_number = row[0]
            sex = row[1]
            age = int(row[2])
            respiratory_rate = float(row[3])
            oxygen_saturation = float(row[4])
            heart_rate = float(row[5])
            systolic_bp = float(row[6])
            temperature = float(row[7])
            alertness = row[8]
            inspired_oxygen = row[9]
            performance_status = row[10]

            total_score = calculate_total_score(
                sex, age, respiratory_rate, oxygen_saturation,
                heart_rate, systolic_bp, temperature, alertness,
                inspired_oxygen, performance_status
            )

            risk_score = calculate_risk_score(total_score)

            hospital_name = find_hospital(age, sex, risk_score)
            hospital_address = get_address(hospital_name)

            patient_info = [
                sex, age, respiratory_rate, oxygen_saturation,
                heart_rate, systolic_bp, temperature, alertness,
                inspired_oxygen, performance_status, risk_score,
                hospital_name, hospital_address
            ]

            if patient_number in result:
                result[patient_number].append(patient_info)
            else:
                result[patient_number] = [patient_info]

    return result


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [8]:
import requests
import json

# Load your JSON data from your local file
with open('your_results.json', 'r') as your_results_file:
    your_json = json.load(your_results_file)

web_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"

# Fetch JSON data from the web URL
response = requests.get(web_url)
if response.status_code == 200:
    web_json = response.json()

    # Compare the two JSON objects
    if your_json == web_json:
        print("The JSON objects are identical. Successful comparison.")
    else:
        print("The JSON objects are not identical.")
else:
    print(f"Failed to retrieve data from the web. Status code: {response.status_code}")


The JSON objects are identical. Successful comparison.


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---